# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [93]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

In [94]:
# Mesh
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

2024-07-27 19:10:41.789 ( 518.053s) [main            ]         graphbuild.cpp:356   INFO| Build local part of mesh dual graph
2024-07-27 19:10:41.789 ( 518.054s) [main            ]           ordering.cpp:204   INFO| GPS pseudo-diameter:(28) 499-0
2024-07-27 19:10:41.789 ( 518.054s) [main            ]           Topology.cpp:1330  INFO| Create topology (single cell type)
2024-07-27 19:10:41.789 ( 518.054s) [main            ]           Topology.cpp:1044  INFO| Create topology (generalised)
2024-07-27 19:10:41.790 ( 518.054s) [main            ]                MPI.cpp:164   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 1
2024-07-27 19:10:41.790 ( 518.054s) [main            ]                MPI.cpp:235   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
2024-07-27 19:10:41.790 ( 518.054s) [main            ]          partition.cpp:233   INFO| Compute ghost indices
2024-07-27 19:10:41.790 ( 518.054s) [main           

We create two python functions for determining the facets to apply boundary conditions to

In [95]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

fdim = domain.topology.dim - 1

print(f"Domain topology = {domain.topology.dim}")
print(f"fdim = {domain.topology.dim - 1}\n")

left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

print(f"Left faces = {left_facets}")
print(f"Right faces = {right_facets}")

Domain topology = 3
fdim = 2

Left faces = [  2   8  14  20  38  44  50  56  86  92  98 104 110 155 161 167 173 179
 185 252 258 264 346 352 440]
Right faces = [1468 1469 1470 1471 1545 1546 1547 1548 1610 1611 1612 1613 1614 1661
 1662 1663 1664 1665 1666 1695 1696 1697 1713 1714 1722]


2024-07-27 19:10:41.814 ( 518.079s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 19:10:41.814 ( 518.079s) [main            ]topologycomputation.cpp:870   INFO| Computing mesh connectivity 2 - 3 from transpose.
2024-07-27 19:10:41.814 ( 518.079s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (0,0)
2024-07-27 19:10:41.814 ( 518.079s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (2,0)
2024-07-27 19:10:41.815 ( 518.079s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (0,0) - (3,0)
2024-07-27 19:10:41.815 ( 518.079s) [main            ]topologycomputation.cpp:870   INFO| Computing mesh connectivity 0 - 3 from transpose.
2024-07-27 19:10:41.815 ( 518.079s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (0,0)
2024-07-27 19:10:41.815 ( 518.079s) [main            ]topologycomputation.cpp:79

Next, we create a  marker based on these two functions

In [96]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

print(f"marked_facets = {marked_facets}")
print(f"marked_values = {marked_values}")
print(f"sorted_facets = {sorted_facets}")
print(f"facet_tag = {facet_tag.indices}") # = marked_facets

marked_facets = [   2    8   14   20   38   44   50   56   86   92   98  104  110  155
  161  167  173  179  185  252  258  264  346  352  440 1468 1469 1470
 1471 1545 1546 1547 1548 1610 1611 1612 1613 1614 1661 1662 1663 1664
 1665 1666 1695 1696 1697 1713 1714 1722]
marked_values = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
sorted_facets = [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
facet_tag = [   2    8   14   20   38   44   50   56   86   92   98  104  110  155
  161  167  173  179  185  252  258  264  346  352  440 1468 1469 1470
 1471 1545 1546 1547 1548 1610 1611 1612 1613 1614 1661 1662 1663 1664
 1665 1666 1695 1696 1697 1713 1714 1722]


We then create a function for supplying the boundary condition on the left side, which is fixed.

In [97]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

print(f"Boundary conditions = {u_bc}")

Boundary conditions = [0. 0. 0.]


To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [98]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
right_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(2))

print(f"left_dofs = {left_dofs}")
print(f"right_dofs = {right_dofs}")

# bcs = [fem.dirichletbc(u_bc, left_dofs, V), fem.dirichletbc(u_bc, right_dofs, V)]
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

left_dofs = [   0    2    4    6    9   10   14   17   22   45   47   49   52   54
   58   63   65   67   69   72   76  123  125  127  130  132  136  141
  143  145  148  153  155  157  159  162  166  245  247  249  252  254
  258  263  265  267  270  275  277  279  282  287  289  291  293  296
  300  419  421  423  426  428  432  437  439  441  444  449  451  453
  456  461  463  465  468  473  475  477  479  482  486  653  655  657
  660  665  667  669  672  677  679  681  684  689  691  693  696  895
  897  899  902  907  909  911  914  919  921  923  926 1149 1151 1153
 1156 1161 1163 1165 1168 1407 1409 1411 1414]
right_dofs = [3839 3840 3841 3842 3844 3845 3847 3849 3853 4081 4082 4083 4085 4086
 4089 4093 4094 4095 4096 4098 4101 4305 4306 4307 4309 4310 4313 4317
 4318 4319 4321 4325 4326 4327 4328 4330 4333 4505 4506 4507 4509 4510
 4513 4517 4518 4519 4521 4525 4526 4527 4529 4533 4534 4535 4536 4538
 4541 4673 4674 4675 4677 4678 4681 4685 4686 4687 4689 4693 4694 4695
 4697

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [99]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [100]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [101]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [102]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
# P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [103]:
P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [104]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [105]:
problem = NonlinearProblem(F, u, bcs)

2024-07-27 19:10:41.972 ( 518.237s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 19:10:41.972 ( 518.237s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)
2024-07-27 19:10:41.972 ( 518.237s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 19:10:41.972 ( 518.237s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)
2024-07-27 19:10:41.980 ( 518.244s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (2,0) - (3,0)
2024-07-27 19:10:41.980 ( 518.244s) [main            ]topologycomputation.cpp:799   INFO| Requesting connectivity (3,0) - (2,0)


and then create and customize the Newton solver

In [106]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


2024-07-27 19:10:42.004 ( 518.268s) [main            ]    SparsityPattern.cpp:385   INFO| Column ghost size increased from 0 to 0


We create a function to plot the solution at each time step.

In [107]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

2024-07-27 19:10:45.080 ( 521.344s) [main            ]      dofmapbuilder.cpp:166   INFO| Checking required entities per dimension
2024-07-27 19:10:45.080 ( 521.344s) [main            ]      dofmapbuilder.cpp:264   INFO| Cell type:0, dofmap:500x27
2024-07-27 19:10:45.080 ( 521.344s) [main            ]      dofmapbuilder.cpp:320   INFO| Global index computation
2024-07-27 19:10:45.080 ( 521.344s) [main            ]      dofmapbuilder.cpp:637   INFO| Got 4 index_maps
2024-07-27 19:10:45.080 ( 521.344s) [main            ]      dofmapbuilder.cpp:644   INFO| Get global indices


Finally, we solve the problem over several time steps, updating the z-component of the traction

In [108]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-07-27 19:10:45.674 ( 521.938s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:10:48.475 ( 524.740s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:10:50.870 ( 527.134s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 8.69535e-07 (tol = 1e-08) r (rel) = 5.24868e-09(tol = 1e-08)
2024-07-27 19:10:50.870 ( 527.134s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 1, Number of iterations 2, Load [ 0.   0.  -1.5]


2024-07-27 19:10:51.324 ( 527.589s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:10:53.961 ( 530.225s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:10:56.324 ( 532.588s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 8.69535e-07 (tol = 1e-08) r (rel) = 5.24868e-09(tol = 1e-08)
2024-07-27 19:10:56.324 ( 532.588s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 2, Number of iterations 2, Load [ 0.  0. -3.]


2024-07-27 19:10:56.724 ( 532.989s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:10:59.434 ( 535.699s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:11:01.753 ( 538.017s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 8.69535e-07 (tol = 1e-08) r (rel) = 5.24867e-09(tol = 1e-08)
2024-07-27 19:11:01.753 ( 538.017s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


...

Time step 7, Number of iterations 2, Load [  0.    0.  -10.5]


2024-07-27 19:11:24.538 ( 560.802s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:11:27.362 ( 563.626s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:11:29.725 ( 565.989s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 8.69535e-07 (tol = 1e-08) r (rel) = 5.24867e-09(tol = 1e-08)
2024-07-27 19:11:29.725 ( 565.989s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 8, Number of iterations 2, Load [  0.   0. -12.]


2024-07-27 19:11:30.149 ( 566.413s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:11:32.897 ( 569.161s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 19:11:35.410 ( 571.675s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 8.69534e-07 (tol = 1e-08) r (rel) = 5.24867e-09(tol = 1e-08)
2024-07-27 19:11:35.411 ( 571.675s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.


Time step 9, Number of iterations 2, Load [  0.    0.  -13.5]
